<a href="https://colab.research.google.com/github/poursa/NNZHExercises/blob/main/Micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3*a**2-1/2*a**(-0.5), 3*cos(3*b) + 2.5*b**1.5, c**(-2)] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell
a = 2
b = 3
c = 4

dfa = (f(a+0.000001,b,c) - f(a,b,c))/0.000001
dfb = (f(a,b+0.000001,c) - f(a,b,c))/0.000001
dfc = (f(a,b,c+0.000001) - f(a,b,c))/0.000001

# -----------
numerical_grad = [dfa, dfb, dfc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [6]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.
a = 2
b = 3
c = 4

dfa = (f(a+0.000001,b,c) - f(a-0.000001,b,c))/(2*0.000001)
dfb = (f(a,b+0.000001,c) - f(a,b-0.000001,c))/(2*0.000001)
dfc = (f(a,b,c+0.000001) - f(a,b,c-0.000001))/(2*0.000001)

# -----------
numerical_grad2 = [dfa, dfb, dfc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

dist1 = [a - b for a , b in zip(ans, numerical_grad) ]
dist2 = [a - b for a , b in zip(ans, numerical_grad2) ]

print(dist1, "\n" ,dist2)


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188
[5.958216721779763e-06, -1.3955669331267018e-06, 1.5256830465659732e-08] 
 [7.599716411732516e-10, -2.9031710369054053e-09, -2.8629187909245957e-10]


## section 2: support for softmax

In [134]:
# Value class starter code, with many functions taken out
from math import exp, log, e

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __rtruediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    return other / self

  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, (self, other), '/')
    
    def _backward():
      self.grad += (1.0 / other.data) * out.grad
      other.grad += (-self.data/(other.data**2)) * out.grad
    out._backward = _backward
    
    return out

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __radd__(self, other): # exactly as in the video
    return self + other
  
  def __mul__(self,other): 
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    
    return out

  def __rmul__ (self, other):
    return other * self
  
  def __neg__(self):
    return self * -1
  
  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1.0 / 2.7 )*(1.0 / self.data) * out.grad
    out._backward = _backward
    return out

  def exp(self): 
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
      self.grad += exp(self.data) * out.grad
    out._backward = _backward
    return out

  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):

  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out



# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()

print(loss)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

Value(data=2.1755153626167147)
WRONG! for dim 0: expected 0.041772570515350445, yours returns 0.015471322413092757
WRONG! for dim 1: expected 0.8390245074625319, yours returns 0.3107498175787155
WRONG! for dim 2: expected 0.005653302662216329, yours returns 0.0020938158008208625
WRONG! for dim 3: expected -0.8864503806400986, yours returns -0.3283149557926292


In [ ]:
import pdb

In [51]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch


In [131]:
plogits = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()
plogits.requires_grad = True
prob = plogits.softmax(0)
loss = -prob[3].log()
loss.backward()
print(plogits.grad)
print(prob)
print(loss)


tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)
tensor([0.0418, 0.8390, 0.0057, 0.1135], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
tensor(2.1755, dtype=torch.float64, grad_fn=<NegBackward0>)
